In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Wählen Sie zwischen den beiden Hausaufgaben eine aus und implementieren Sie diese, gern auch als Kleingruppe.

## Aufgabe 1



Implementieren Sie ein Verfahren zur Berechnung von Ähnlichkeiten zwischen Wörtern basierend auf der Distributional Hypothesis, indem Sie für jedes Wort im Vokabular zählen, wie häufig die anderen Wörter in einer  Umgebung fixer Länge vorkommen (siehe Vorlesungsmaterial). Sie können eines der folgenden Korpora oder einen eigens gewählten Korpus verwenden:

1. Kochrezepte (`recipes.json`)
2. Movie-Reviews (`movies.csv`)

Führen Sie hierfür die aus Ihrer Sicht **notwendigen** Schritte durch, mindestens jedoch:

- a) Tokenisierung
- b) Ggf. Wort-Normalisierung (z.B. Lowercasing, Lemmatisierung, usw.) 
- c) Term-Term-Matrix (basierend auf einfacher Zählung von Vorkommnissen). Hier wählen Sie selbst, was Sie als **Umgebung/Kontext eines Wortes** betrachten. Es kann das ganze Dokument sein, der Satz, in dem das Wort vorkommt, oder 2 (3 oder 4) Wörter davor und danach. Sie können auch mehrere Varianten ausprobieren.
- e) Wählen Sie anschließend **5 Terme** aus und und ermitteln jeweils die **10 ähnlichsten Terme** basierend auf der Term-Term-Matrix und der Kosinus-Distanz. 

In [4]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/SoSe2023/B45 Neuronale Netze und Deep Learning/week_6/movies.csv', header=None, names=['review', 'sentiment'])

In [5]:
# Tokenisierung des Textes in einzelne Wörter

import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize

df['tokens'] = df['review'].apply(lambda x: word_tokenize(x.lower()))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
# Normalisierung der Wörter durch Lemmatisierung

nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

df['tokens'] = df['tokens'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [2]:
# Erstellung einer Term-Term-Matrix auf der Grundlage einer Umgebung mit fester Länge. 
# Hier verwenden wir ein Fenster von 5 Wörtern (2 Wörter vor und nach jedem Zielwort)

from collections import defaultdict
import numpy as np

def create_term_term_matrix(docs, window_size=5):
    vocab = set()
    cooccur = defaultdict(lambda: defaultdict(int))
    for doc in docs:
        for i, word in enumerate(doc):
            vocab.add(word)
            for j in range(max(0, i-window_size), min(i+window_size+1, len(doc))):
                if i != j:
                    cooccur[word][doc[j]] += 1

    vocab = sorted(vocab)
    term_term_matrix = np.zeros((len(vocab), len(vocab)))
    for i, term_i in enumerate(vocab):
        for j, term_j in enumerate(vocab):
            if i == j:
                term_term_matrix[i, j] = 1.0
            else:
                term_term_matrix[i, j] = cooccur[term_i][term_j] + cooccur[term_j][term_i]

    return term_term_matrix, vocab

In [6]:
# Erstellung einer Term-Term-Matrix auf der Grundlage der tokenisierten Bewertungen
term_term_matrix, vocab = create_term_term_matrix(df['tokens'])

KeyboardInterrupt: ignored

In [ ]:
# Berechnung der Kosinusähnlichkeit zwischen jedem Begriffspaar im Vokabular anhand der Term-Term-Matrix
from sklearn.metrics.pairwise import cosine_similarity
similarity_matrix = cosine_similarity(term_term_matrix)

In [ ]:
# Auswahl von 5 Zielbegriffen und Ermittlung der 10 ähnlichsten Begriffe für jeden auf der Grundlage der Kosinus-Ähnlichkeitsmatrix

target_terms = ['good', 'bad', 'great', 'awful', 'love']
num_similar_terms = 10

for term in target_terms:
    term_index = vocab.index(term)
    similar_terms = [(vocab[i], similarity_matrix[term_index, i]) for i in range(len(vocab)) if vocab[i] != term]
    similar_terms = sorted(similar_terms, key=lambda x: x[1], reverse=True)[:num_similar_terms]
    print(f"Similar terms for '{term}':")
    for similar_term in similar_terms:
        print(f"{similar_term[0]} ({similar_term[1]:.2f})")
    print()

## Aufgabe 2

Implementieren Sie Shannon's Methode basierend auf Bigrammen aus der Vorlesung anhand eines frei wählbaren Korpus. Wenn möglich schreiben Sie Ihren Code so, dass man leicht auch Trigramme oder beliebige N-Gramme statt Bigrammen nutzen kann. Implementieren Sie ebenfalls eine Funktion, mit welcher Text generiert werden kann. Der Text soll so lange generiert werden, bis das Satz-Ende Symbol erzeugt wird, oder bis eine vorgegebene textlänge erreicht wird.


### Implementation of Shannon's method based on bigrams using NLTK and the movie reviews dataset

In [ ]:
import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize
from collections import defaultdict
import random

In [19]:
# Load the movie reviews dataset
with open('/content/drive/MyDrive/SoSe2023/B45 Neuronale Netze und Deep Learning/week_6/movies.csv', 'r', encoding='utf-8') as f:
    reviews = f.readlines()

# Tokenize the reviews into individual words
reviews = [word_tokenize(review.strip().lower()) for review in reviews]

In [22]:
# Create a frequency distribution of bigrams
bigrams = defaultdict(lambda: defaultdict(int))
for review in reviews:
    for i in range(len(review)-1):
        bigrams[review[i]][review[i+1]] += 1

# Compute the conditional probabilities of each word given its predecessor
for word in bigrams:
    total_count = sum(bigrams[word].values())
    for next_word in bigrams[word]:
        bigrams[word][next_word] /= total_count

# Define a function to generate text based on the bigram model
def generate_text(start_word, max_length=None):
    text = [start_word]
    current_word = start_word
    length = 0
    while True:
        next_word = random.choices(list(bigrams[current_word].keys()), list(bigrams[current_word].values()))[0]
        text.append(next_word)
        current_word = next_word
        length += 1
        if next_word == '.' or (max_length is not None and length >= max_length):
            break
    return ' '.join(text)

# Generate a sample sentence starting with the word "the"
print(generate_text('the'))

i actually happened , so karla , believe that did to suck .


### Trigrams or arbitrary N-grams

In [17]:
# Create a frequency distribution of trigrams
trigrams = defaultdict(lambda: defaultdict(int))
for review in reviews:
    for i in range(len(review)-2):
        trigrams[(review[i], review[i+1])][review[i+2]] += 1

# Compute the conditional probabilities of each word given its two predecessors
for word_pair in trigrams:
    total_count = sum(trigrams[word_pair].values())
    for next_word in trigrams[word_pair]:
        trigrams[word_pair][next_word] /= total_count

# Define a function to generate text based on the trigram model
def generate_text(start_word_pair, max_length=None):
    text = list(start_word_pair)
    current_word_pair = start_word_pair
    length = 0
    while True:
        next_word = random.choices(list(trigrams[current_word_pair].keys()), list(trigrams[current_word_pair].values()))[0]
        text.append(next_word)
        current_word_pair = tuple(text[-2:])
        length += 1
        if next_word == '.' or (max_length is not None and length >= max_length):
            break
    return ' '.join(text)

# Generate a sample sentence starting with the words "the movie"
print(generate_text(('the', 'movie')))

the movie is a precious performance by alyssa milano is something i 've seen a movie so much more depressing things happen .


In [32]:
# Create a frequency distribution of 4-grams
fivegrams = defaultdict(lambda: defaultdict(int))
for review in reviews:
    for i in range(len(review)-3):
        fivegrams[(review[i], review[i+1], review[i+2])][review[i+3]] += 1

# Compute the conditional probabilities of each word given its four predecessors
for word_tuple in fivegrams:
    total_count = sum(fivegrams[word_tuple].values())
    for next_word in fivegrams[word_tuple]:
        fivegrams[word_tuple][next_word] /= total_count

# Define a function to generate text based on the 4-gram model
def generate_text(start_word_tuple, max_length=None):
    text = list(start_word_tuple)
    current_word_tuple = start_word_tuple
    length = 0
    while True:
        if current_word_tuple not in fivegrams:
            current_word_tuple = random.choice(list(fivegrams.keys()))
            text = list(current_word_tuple)
        next_word = random.choices(list(fivegrams[current_word_tuple].keys()), list(fivegrams[current_word_tuple].values()))[0]
        text.append(next_word)
        current_word_tuple = tuple(text[-3:])
        length += 1
        if next_word == '.' or (max_length is not None and length >= max_length):
            break
    return ' '.join(text)

# Generate a sample sentence starting with the words "the movie was really"
print(generate_text(('the', 'movie', 'was')))

the movie was originally conceived as part of the film : some violent confrontations involving the angels , also gives us a great love story told with the disaster as a backdrop and a portrayal of one of the best of 1998 .
